In [29]:
import random
import numpy as np
random.seed(10)

def loss(y_true, y_pred):
    return np.sum((y_true - y_pred)**2)

def loss_d(y_true, y_pred):
    return 2 * y_pred - 2 * y_true

def sigm_fun(x):
    return 1 / (1 + np.exp(-x))
def sigm_d(sigm_val):
    return sigm_val * (1 - sigm_val)

class Neuron:
    def __init__(self, input_dim, activation_f = sigm_fun, activation_f_d = sigm_d):
        self.activation_f = activation_f
        self.activation_f_d = activation_f_d
        weight_list = []
        for i in range(input_dim):
            weight_list.append(random.uniform(-1, 1))
        self.weights = np.asarray(weight_list)
        self.b = random.uniform(-1, 1)
        self.b_grad = 0
        self.w_grad = np.zeros(input_dim)
        self.out_grad = 0
    
    def work(self, inputs):
        self.x = np.array(inputs)
        sum = np.sum(inputs * self.weights) + self.b
        self.output = self.activation_f(sum)
        return self.output

    def x_grad(self, i):
        return self.out_grad * self.activation_f_d(self.output) * self.weights[i]

    def generate_param_grad(self):
        self.w_grad = self.out_grad * self.activation_f_d(self.output) * self.x
        self.b_grad = self.out_grad * self.activation_f_d(self.output)
            

class NeuralNet:
    def __init__(self, number_of_neurons_in_layer, number_of_layers, input_dim):
        self.input_layer = []
        self.neurons: list[Neuron] = []
        for i in range(number_of_neurons_in_layer):
            neuron = Neuron(input_dim)
            self.input_layer.append(neuron)
            self.neurons.append(neuron)
        self.net = []
        for i in range(number_of_layers - 1):
            current_layer =[]
            for j in range(number_of_neurons_in_layer):
                neuron = Neuron(number_of_neurons_in_layer)
                current_layer.append(neuron)
                self.neurons.append(neuron)
            self.net.append(current_layer)
        neuron = Neuron(number_of_neurons_in_layer, activation_f=lambda x:x, activation_f_d=lambda x:1)
        self.output_layer = [neuron]
        self.neurons.append(neuron)
                                     
    def predict(self, inputs):
        temp_inputs = []
        for neuron in self.input_layer:
            temp_inputs.append(neuron.work(inputs))
        temp_inputs2 = []
        for layer in self.net:
            for neuron in layer:
                temp_inputs2.append(neuron.work(temp_inputs))
            temp_inputs = temp_inputs2
            temp_inputs2 = []
        return self.output_layer[0].work(temp_inputs)

    def backprop_single_input(self, input, expected_value):
        y_pred = self.predict(input)
        error = loss(y_pred, expected_value)
        error_d = loss_d(y_pred=y_pred, y_true=expected_value)
        self.output_layer[0].out_grad = error_d
        self.output_layer[0].generate_param_grad()
        next_layer = self.output_layer
        layers = self.net.copy()
        layers.append(self.input_layer)
        for layer in reversed(layers):
            for i, neuron in enumerate(layer):
                neuron.w_grad = 0
                neuron.out_grad = 0
                neuron.b_grad = 0
                for next_neuron in next_layer:
                    neuron.out_grad += next_neuron.x_grad(i)
                neuron.generate_param_grad()
        return error
    
    def train(self, X, Y, learning_rate=0.05, max_iter = 100):
        tot_error = 0
        n = len(X)
        tot_grad = [(neuron.w_grad, neuron.b_grad) for neuron in self.neurons]

        for k in range(max_iter):
            for x, y in zip(X, Y):
                tot_error+=self.backprop_single_input(x, y)
                print(tot_error)
                tot_grad = [(grad[0] + neuron.w_grad, grad[1] + neuron.b_grad) 
                            for grad, neuron in zip(tot_grad, self.neurons)]
            tot_error = tot_error/n
            print(f"Mean error {tot_error}")
            for grad, neuron in zip(tot_grad, self.neurons):
                neuron.weights += -grad[0] / n * learning_rate
                neuron.b += -grad[1] / n * learning_rate


In [30]:
random.seed(10)
net = NeuralNet(number_of_layers=3,number_of_neurons_in_layer=3,input_dim=3)


x = [2,3,6]
y = 10
net.train([x], [y])
net.predict(x)

92.71062075495081
Mean error 92.71062075495081
157.07073751827124
Mean error 157.07073751827124
183.2963328393177
Mean error 183.2963328393177
184.03280505081324
Mean error 184.03280505081324
200.92376879793082
Mean error 200.92376879793082
257.51380308061124
Mean error 257.51380308061124
300.71181507586755
Mean error 300.71181507586755
317.59014461143835
Mean error 317.59014461143835
328.0601981288117
Mean error 328.0601981288117
332.7220466764819
Mean error 332.7220466764819
334.8267938655113
Mean error 334.8267938655113
335.4848739461601
Mean error 335.4848739461601
335.5052803353272
Mean error 335.5052803353272
335.7794494653469
Mean error 335.7794494653469
337.05726576073675
Mean error 337.05726576073675
339.6817227801722
Mean error 339.6817227801722
343.46509615846196
Mean error 343.46509615846196
347.76684810633486
Mean error 347.76684810633486
351.74545250560544
Mean error 351.74545250560544
354.6871466043694
Mean error 354.6871466043694
356.2841596938383
Mean error 356.2841596

C:\Users\Filip\AppData\Local\Temp\ipykernel_12856\1397787267.py:12: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


np.float64(9.22539705431861)